In [3]:
from jbi100_app.views.menu import make_menu_layout
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from dash import html, dcc
from jbi100_app.main import app
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Import from modified_data.py
from jbi100_app.modified_data import load_team_data, load_player_data, top_scoring_teams, best_defensive_teams, top_scoring_players

# File paths for the CSV files
team_data_csv = 'team_data.csv'
player_data_csv = 'Player_stats.csv'
match_data_csv = 'match_data.csv'  # Assuming the match data file is named this

# Function to process team data
def process_team_data(selected_team):
    match_data = pd.read_csv(match_data_csv)
    team_data = match_data[(match_data['home_team'] == selected_team) | (match_data['away_team'] == selected_team)]
    return team_data

# Function to create team performance chart
def calculate_team_performance(team, match_data):
    wins = draws = losses = goals_scored = goals_against = 0

    for _, row in match_data.iterrows():
        home_goals, away_goals = map(int, row['score'].split('–'))
        
        if row['home_team'] == team:
            goals_scored += home_goals
            goals_against += away_goals
            if home_goals > away_goals:
                wins += 1
            elif home_goals == away_goals:
                draws += 1
            else:
                losses += 1

        elif row['away_team'] == team:
            goals_scored += away_goals
            goals_against += home_goals
            if away_goals > home_goals:
                wins += 1
            elif away_goals == home_goals:
                draws += 1
            else:
                losses += 1

    return wins, draws, losses, goals_scored, goals_against

def create_team_performance_chart(team_data, selected_team):
    team_performance = calculate_team_performance(selected_team, team_data)
    team_performance_df = pd.DataFrame({
        'Statistic': ['Wins', 'Draws', 'Losses', 'Goals Scored', 'Goals Against'],
        'Total': team_performance
    })

    fig = px.bar(team_performance_df, x='Statistic', y='Total', title=f'Overall Team Performance for {selected_team}')
    return fig

# Function to create average specifics per match chart
def create_average_specifics_chart(team_data):
    # Selecting relevant columns for the radar chart
    columns_of_interest = ['offsides', 'goal_kicks', 'throw_ins', 'long_balls', 'tackles', 'interceptions', 'clearances', 'fouls', 'corners', 'crosses', 'possession_percentage', 'completed_passes', 'attempted_passes', 'shots_on_target', 'total_shots', 'saves']
    average_data = team_data[columns_of_interest].mean()

    fig = go.Figure(data=go.Scatterpolar(
        r=average_data.values,
        theta=average_data.index,
        fill='toself'
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, average_data.max()]
            )),
        title='Average Specifics per Match'
    )

    return fig

# Callback for updating the dashboard
@app.callback(
    Output('right-column', 'children'),
    [Input('select-team-button', 'n_clicks')],
    [State('team-selector', 'value')]
)
def update_team_dashboard(n_clicks, selected_team):
    if n_clicks is None:
        raise PreventUpdate

    # Load match data and filter for the selected team
    match_data = pd.read_csv(match_data_csv)
    team_data = match_data[(match_data['home_team'] == selected_team) | (match_data['away_team'] == selected_team)]

    # Create visualizations
    team_performance_chart = create_team_performance_chart(team_data, selected_team)
    average_specifics_chart = create_average_specifics_chart(team_data)

    # Create new layout with these charts
    new_layout = html.Div([
        html.H2(f"{selected_team} Dashboard"),
        dcc.Graph(figure=team_performance_chart),
        dcc.Graph(figure=average_specifics_chart)
    ])

    return new_layout

if __name__ == '__main__':
    # Load data
    team_data = load_team_data(team_data_csv)
    player_data = load_player_data(player_data_csv)

    # Create bar charts
    top_teams_chart = top_scoring_teams(team_data)
    best_defensive_teams_chart = best_defensive_teams(team_data)
    top_players_chart = top_scoring_players(player_data)

    app.layout = html.Div(
        id="app-container",
        children=[
            html.Div(
                id="left-column",
                className="three columns",
                children=make_menu_layout()
            ),
            html.Div(
                id="right-column",
                className="nine columns",
                children=[
                    html.H1("Team Performance Dashboard"),
                    html.H2("General Home Page"),
                    html.Div(id='top-scoring-teams', children=[
                        html.H3("Top 10 Scoring Teams"),
                        dcc.Graph(figure=top_teams_chart)
                    ]),
                    html.Div(id='best-defensive-teams', children=[
                        html.H3("Top 10 Defensive Teams"),
                        dcc.Graph(figure=best_defensive_teams_chart)
                    ]),
                    html.Div(id='top-scoring-players', children=[
                        html.H3("Top 10 Scoring Players"),
                        dcc.Graph(figure=top_players_chart)
                    ])
                ],
            ),
        ],
    )

    app.run_server(debug=False, dev_tools_ui=False)